In [1]:
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import bz2
import json
from urllib.parse import urlparse
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [7]:
#creation d'un nouveau fichier avec une nouvelle colonne pour le domaine (cf colab)
from tld import get_tld

def get_domain(url):
    res = get_tld(url, as_object=True)
    return res.tld

In [8]:
#pareil
path_to_file = 'quotes-2020.json.bz2' 
path_to_out = 'quotes-2020-domains.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            urls = instance['urls'] # extracting list of links
            domains = []
            for url in urls:
                tld = get_domain(url)
                domains.append(tld)
            instance['domains'] = domains # updating the sample with domain name
            d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

In [2]:
#cette fois ci on rajoute une colonne avec simplement le nom du site
path_to_file = 'quotes-2020-domains.json.bz2' 
path_to_out = '2020a.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
    with bz2.open(path_to_out, 'wb') as d_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            urls = instance['urls'] # extracting list of links
            website = []
            for url in urls:
                net = urlparse(url)
                neto=net.netloc
                website.append(neto)
            instance['website'] = website # updating the sample with domain name
            d_file.write((json.dumps(instance)+'\n').encode('utf-8')) # writing in the new file

In [ ]:
#print de cette colonne
path_to_file = '2020a.json.bz2'

with bz2.open(path_to_file, 'rb') as s_file:
        for instance in s_file:
            instance = json.loads(instance) # loading a sample
            print(instance['website']) # extracting list of links   

In [2]:
#on chun le dataset nouvellement créé
def process_chunk(chunk):
        print(f'Processing chunk with {len(chunk)} rows')
        print(chunk.columns)
for chunk in pd.read_json('2020a.json.bz2', lines=True, compression='bz2', chunksize=250000, encoding='utf-8'):
    process_chunk(chunk)

Processing chunk with 250000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains', 'website'],
      dtype='object')
Processing chunk with 250000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains', 'website'],
      dtype='object')
Processing chunk with 250000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains', 'website'],
      dtype='object')
Processing chunk with 250000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains', 'website'],
      dtype='object')
Processing chunk with 250000 rows
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase', 'domains', 'website'],
      dtype='object')
Processing chunk with 250000 rows
Index(['quoteID', 'qu

In [40]:
lis=chunk['website'].value_counts().to_frame()
lis.head(20)

,website
['www.msn.com'],1202
['express.co.uk'],1184
['www.cheatsheet.com'],925
['www.bostonglobe.com'],769
['www.foxnews.com'],763
['timesofindia.indiatimes.com'],745
['www.dailystar.co.uk'],742
['www.telegraph.co.uk'],741
['indianexpress.com'],741
['www.thesun.co.uk'],730


In [15]:
chunk['speaker'].value_counts().to_frame()

,speaker
None,84128
President Donald Trump,1258
Bernie Sanders,595
Joe Biden,581
President Trump,546
...,...
Kirstjen Nielsen,1
Jennifer Anderson,1
alan tongue,1
Wes Welker,1


In [3]:
chunk['website']=chunk['website'].astype('str')

In [52]:
#creation d'un dataframe pour quelques journaux
fox=chunk[chunk['website'].str.contains("www.foxnews.com")]
ny=chunk[chunk['website'].str.contains("www.nytimes.com")]
brei=chunk[chunk['website'].str.contains("www.breitbart.com")]
cnn=chunk[chunk['website'].str.contains("cnn.com")]
guard=chunk[chunk['website'].str.contains("www.theguardian.com")]
slate=chunk[chunk['website'].str.contains("slate.com")]
buzz=chunk[chunk['website'].str.contains("buzzfeed.com")]

In [53]:
fox_quotes=fox['quotation']
ny_quotes=ny['quotation']
brei_quotes=brei['quotation']
cnn_quotes=cnn['quotation']
guard_quotes=guard['quotation']
slate_quotes=slate['quotation']
buzz_quotes=buzz['quotation']

In [6]:
sia = SentimentIntensityAnalyzer()

In [8]:
fox.shape

(1787, 11)

In [18]:
ny.shape

(1284, 11)

In [22]:
brei.shape

(3708, 11)

In [35]:
cnn.shape

(1446, 11)

In [43]:
guard.shape

(687, 11)

In [47]:
slate.shape

(123, 11)

In [54]:
buzz.shape

(279, 11)

In [16]:
#analyse de sentiments positif et negatif pour les journaux précédemment choisis
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in fox_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/1787
average_neg=neg/1787
print(average_pos,average_neg)

0.11182372691661988 0.07998601007274761


In [19]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in ny_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/1284
average_neg=neg/1284
print(average_pos,average_neg)

0.10962928348909654 0.07516510903426793


In [23]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in brei_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/3708
average_neg=neg/3708
print(average_pos,average_neg)

0.12019363538295594 0.0813460086299893


In [38]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in cnn_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/1446
average_neg=neg/1446
print(average_pos,average_neg)

0.113540110650069 0.06954356846473027


In [44]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in guard_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/687
average_neg=neg/687
print(average_pos,average_neg)

0.11037845705967979 0.07798107714701595


In [48]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in slate_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/123
average_neg=neg/123
print(average_pos,average_neg)

0.09873170731707316 0.07367479674796752


In [55]:
pos=0
neg=0
average_pos=0
average_neg=0
for quotation in buzz_quotes:
    result=sia.polarity_scores(quotation)
    pos=pos+result["pos"]
    neg=neg+result["neg"]
average_pos=pos/279
average_neg=neg/279
print(average_pos,average_neg)

0.1609749103942653 0.06458064516129029
